In [2]:
# Load libraries 
import os
import re
import time
import openai 
import pandas as pd 
from tqdm import tqdm
from typing import Tuple
from dotenv import load_dotenv


load_dotenv()
pd.set_option("display.max_columns", None)

In [3]:
# Setting credentials
OPENAI_KEY = os.getenv("OPENAI_API_KEY", default = None) 
openai.api_key = OPENAI_KEY

In [5]:
# Load dataset
data = pd.read_excel("../dataset/Full Dataset Kotor updated 2.0.xlsx")
data.head()

,created_at,id_str,full_text,quote_count,reply_count,retweet_count,favorite_count,lang,user_id_str,conversation_id_str,username,tweet_url
0,Fri Nov 17 23:58:21 +0000 2023,1.725665e+18,@gogo74070675957 @iina_surbakti @gibran_tweet ...,0.0,0.0,0.0,1.0,in,1.213867e+18,1.725499e+18,ArjunaOnthesky,https://twitter.com/ArjunaOnthesky/status/1725...
1,Fri Nov 17 23:58:19 +0000 2023,1.725665e+18,@vendie7 Btw sy msh ingat omelanmu ttg prof MD...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fri Nov 17 23:57:49 +0000 2023,1.725665e+18,ðŸ”´âšªï¸ PEMILU terutama PILPRES adalah SATU...,1.0,7.0,12.0,29.0,in,1.378303e+18,1.725665e+18,_BungHerwin,https://twitter.com/_BungHerwin/status/1725664...
3,Fri Nov 17 23:57:34 +0000 2023,1.725665e+18,@ekagumilars Indonesia aman &amp damai tanpa ...,0.0,0.0,0.0,0.0,in,2.537213e+09,1.725384e+18,irfandjay,https://twitter.com/irfandjay/status/172566450...
4,Fri Nov 17 23:57:31 +0000 2023,1.725664e+18,Pilpres kali ini,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Data Duplicate checking
data.duplicated(subset = ['full_text', 'id_str', 'retweet_count']).value_counts()

False    1564
True      172
dtype: int64

In [7]:
# Overview duplicated data
data[data.duplicated(subset = ['full_text', 'id_str', 'retweet_count'])].head(10)

,created_at,id_str,full_text,quote_count,reply_count,retweet_count,favorite_count,lang,user_id_str,conversation_id_str,username,tweet_url
151,Fri Nov 17 21:00:46 +0000 2023,1.725620e+18,@NenkMonica @gibran_tweet Salahnya dmn dateng ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
152,Fri Nov 17 20:49:46 +0000 2023,1.725617e+18,Kreatif balon #GanjarMahfud2024 anti di Bongka...,0.0,3.0,7.0,24.0,in,7.945097e+17,1.725617e+18,Jayabay19479190,https://twitter.com/Jayabay19479190/status/172...
352,Fri Nov 17 16:24:41 +0000 2023,1.725551e+18,@DPP_PKB @ninikwafiroh Maju bersama AMIN memen...,0.0,0.0,0.0,0.0,in,1.618235e+18,1.725482e+18,IndahRahaleb,https://twitter.com/IndahRahaleb/status/172555...
353,Fri Nov 17 16:23:57 +0000 2023,1.725550e+18,@triwul82 Mahfud MD mengingatkan bahwa tanggun...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
529,Fri Nov 17 23:56:55 +0000 2023,1.725664e+18,@Mdy_Asmara1701 Para Kabinda di banyak daerah ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
586,Fri Nov 17 23:58:21 +0000 2023,1.725665e+18,@gogo74070675957 @iina_surbakti @gibran_tweet ...,0.0,0.0,0.0,1.0,in,1.213867e+18,1.725499e+18,ArjunaOnthesky,https://twitter.com/ArjunaOnthesky/status/1725...
587,Fri Nov 17 23:56:33 +0000 2023,1.725664e+18,Ini Daftar Nama dan Struktur Lengkap TKN Prabo...,0.0,0.0,0.0,0.0,in,1.110933e+18,1.725664e+18,fathw25,https://twitter.com/fathw25/status/17256642523...
588,Fri Nov 17 23:53:50 +0000 2023,1.725664e+18,Kecuali Jokowi jadi ketum Golkar dan Gibran ja...,0.0,0.0,0.0,0.0,in,8.486730e+07,1.725664e+18,tualang,https://twitter.com/tualang/status/17256635676...
589,Fri Nov 17 23:35:18 +0000 2023,1.725659e+18,Temukan dan dapatkan Kaos Baju prabowo gemoy -...,0.0,0.0,0.0,0.0,in,2.993486e+09,1.725659e+18,aris_jenang,https://twitter.com/aris_jenang/status/1725658...
590,Fri Nov 17 23:35:18 +0000 2023,1.725659e+18,Masyarakat Menilai Gibran Tidak Mempunyai Kapa...,0.0,0.0,0.0,1.0,in,1.618289e+18,1.725659e+18,Liza16144812,https://twitter.com/Liza16144812/status/172565...


In [8]:
# Duplicate data filtering
data = data.drop_duplicates(subset = ['full_text', 'id_str', 'retweet_count'])

In [9]:
# Data Duplicate checking - validation
data.duplicated(subset = ['full_text', 'id_str', 'retweet_count']).value_counts()

False    1564
dtype: int64

In [10]:
# Define prompt and ingestion script
def prompt_enrichment(tweet_comment: str) -> str:
    prompt = \
    f"""
    Ekstraksi informasi yang dibutuhkan berdasarkan komentar twitter dibawah, dengan response cukup sesuai yang di definisikan tanpa penjelasan tambahan.

    komentar_twitter: "{tweet_comment}"

    Untuk response cukup isi dengan format dibawah.
    named_entity_recognition: [Jawaban anda: cakupan NER sesuai label "PERSON" atau "ORGANIZATION" saja]
    aspect_sentiment: [Identifikasi verb / noun-phrase hasil dari part-of-speech di dalam komentar, disertai dengan nilai sentiment masing-masing aspect dengan format <aspect (sentiment)>]
    """
    return prompt

def ingest_openai(tweet_comment: str, model_base: str = "gpt-3.5-turbo") -> Tuple[str, int]: 
    token_usage = 0
    response_extraction = ""
    try:
        response = openai.ChatCompletion.create(
            model = model_base, 
            messages = [{"role" : "user", "content" : prompt_enrichment(tweet_comment)}], 
            temperature = 0.1, max_tokens = 512, top_p = 1.0, 
            frequency_penalty = 0.0, presence_penalty = 0.0
        )
        response_extraction = response["choices"][0]["message"]["content"]
        token_usage = response["usage"]["total_tokens"]
    except Exception as E:
        print(f"[ERROR] - {E}")
        print("Retry with Recursive Func")
        time.sleep(5)
        ingest_openai(tweet_comment = tweet_comment)
    return response_extraction, token_usage

In [13]:
# Check total rows
print(f"Total Rows: {data.shape[0]}")

Total Rows: 1564


In [11]:
# Test ingestion
comment = data['full_text'].sample(1).values[0]
extraction, token_usage = ingest_openai(tweet_comment = comment)
print(f"[COMMENT]\n{comment}\n[RESULT - Token Usage: {token_usage}]\n{extraction}")

[COMMENT]
Komisi Pemilihan Umum (KPU) RI telah menetapkan nomor urut terhadap tiga calon presiden dan wakil presiden di Pilpres 2024.  #BersamaIndonesiaMaju #PrabowoGibranIstimewa #PrabowoGemoy Prabowo Subianto
[RESULT - Token Usage: 279]
named_entity_recognition: ["Komisi Pemilihan Umum (KPU) RI", "Prabowo Subianto"]
aspect_sentiment: ["menetapkan nomor urut (positive)", "tiga calon presiden dan wakil presiden (neutral)"]


In [14]:
# Apply on entire dataset
final_result_extraction, final_token_usage = [], []

In [15]:
# Iter and push into array
for comment in tqdm(data["full_text"], desc = "Ingestion Start"):
    result, token = ingest_openai(tweet_comment = comment)
    final_result_extraction.append(result)
    final_token_usage.append(token)

Ingestion Start:   1%|▋                                                            | 19/1564 [07:40<6:43:15, 15.66s/it]

[ERROR] - Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600)
Retry with Recursive Func


Ingestion Start:   2%|█▍                                                          | 36/1564 [29:06<18:26:39, 43.46s/it]

[ERROR] - Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600)
Retry with Recursive Func


Ingestion Start:   4%|██▍                                                          | 63/1564 [44:38<6:37:56, 15.91s/it]

...

Ingestion Start:  98%|████████████████████████████████████████████████████████▋ | 1530/1564 [17:19:09<12:27, 21.99s/it]

[ERROR] - Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600)
Retry with Recursive Func


Ingestion Start:  99%|█████████████████████████████████████████████████████████▎| 1547/1564 [17:31:34<02:05,  7.38s/it]

[ERROR] - Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600)
Retry with Recursive Func


Ingestion Start: 100%|██████████████████████████████████████████████████████████| 1564/1564 [17:44:08<00:00, 40.82s/it]


In [16]:
len(final_result_extraction), len(final_token_usage)

(1564, 1564)

In [19]:
# Assign result into dataframe
data['result extraction'] = final_result_extraction
data['token usage'] = final_token_usage

In [20]:
# Save into dataframe
data.to_csv("../dataset/data_twitter_pemilu_2024_enrich [V2].csv", index = False)